In [1]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
from mysql.connector import (connection)
from collections import defaultdict
cnx = connection.MySQLConnection(user='username', password='password',
                                 host='127.0.0.1',
                                 database='database')


In [2]:
keeper = []
truecar = defaultdict(list)
vehicle = input('what car are u looking for?(brand and model): ').split(' ')

try:
    res = requests.get('https://www.truecar.com/used-cars-for-sale/listings/%s/%s/' % (vehicle[0],vehicle[1],))

    if res.status_code==403:
        raise Exception('\nSorry, the website you are trying to send REST API request is forbbiden 403 Error.')

except Exception as e:
    print("Sorry,something wnet wrong ...!" , e)


else:
    if res.status_code==200:
        print('API request successfully accepted.')
    
    for i in range (1,10):
        
        res = requests.get('https://www.truecar.com/used-cars-for-sale/listings/%s/%s/?page=%i' % (vehicle[0],vehicle[1],i))
        soup = BeautifulSoup(res.text,'html.parser')
        cars = soup.find_all('div',attrs = {'data-test':"cardContent", 'class':"card-content order-3 vehicle-card-body"})
        
        for car in cars:
            want = re.findall(r'\w*?(\d+ \w+ \w+) .* list price(\$\d+\,\d{3})*(\$\d+\,\d{3})(\d+\,*\d*)',car.text)
            if want==[]:
                continue
            final = []
            for i in want:
                for j in i:
                    if j!='':
                        final.append(j)
            if len(final)==4:
                final.remove(final[1])    
            
            all = final[0].split(' ')
            year = all[0]
            year = int(year)
            brand = all[1]
            model = all[2]
            price = final[1]
            mile = final[2]
            if final in keeper:
                continue
            else:
                keeper.extend(final)
        
            cursor = cnx.cursor()
            cursor.execute('INSERT INTO cars VALUES (\'%s\',\'%s\' , \'%i\' , \'%s\' , \'%s\') '  % ((brand,model,year,price,mile)))
            cnx.commit()        
            print('%s %s %i is %s and mileage is %s'%(brand,model,year,price,mile))


        cursor = cnx.cursor()    
        query = 'SELECT * FROM cars WHERE brand = brand or model = model;'    
        cursor.execute(query)
    
            for brand,model,year,price,mile in cursor:
                mile = mile.split(',')
                mile = ''.join(mile)
                mile = int(mile)
                price = price.split('$')
                price = price[1].split(',')
                price = ''.join(price)
                price = int(price)
                truecar["brand"].append(brand)
                truecar["model"].append(model)
                truecar["year"].append(year)
                truecar["price$"].append(price)
                truecar["mileage"].append(mile)

    
    removal = 'TRUNCATE TABLE cars;'    
finally:    
    cnx.close() 
    print('revale')

df = pd.DataFrame(truecar)
print(df)

what car are u looking for?(brand and model):  bmw i3


API request successfully accepted.
BMW i360 2015 is $9,774 and mileage is 89,366
BMW i394 2018 is $16,997 and mileage is 52,454
BMW i394 2017 is $13,998 and mileage is 53,799
BMW i394 2017 is $17,998 and mileage is 41,244
BMW i360 2016 is $16,998 and mileage is 44,784
BMW i394 2018 is $15,793 and mileage is 77,718
BMW i360 2014 is $13,599 and mileage is 79,291
BMW i360 2015 is $10,994 and mileage is 58,569
BMW i360 2014 is $11,925 and mileage is 58,387
BMW i360 2014 is $9,399 and mileage is 80,929
BMW i394 2017 is $17,400 and mileage is 15,092
BMW i3s 2021 is $25,900 and mileage is 62,813
BMW i360 2015 is $14,338 and mileage is 45,049
BMW i360 2016 is $9,800 and mileage is 101,928
BMW i360 2015 is $15,998 and mileage is 51,969
BMW i360 2015 is $9,900 and mileage is 77,832
BMW i360 2015 is $11,955 and mileage is 77,833
BMW i360 2016 is $16,998 and mileage is 44,235
BMW i360 2016 is $12,990 and mileage is 67,642
BMW i360 2016 is $13,999 and mileage is 46,651
BMW i360 2015 is $10,498 and 

In [10]:
filt = (df['brand']=='BMW') & (df['year'] > 2020)
new_df = df.loc[filt]
new_df

,Brand,Model,Year,Price$,Mileage
40,BMW,s,2021,25900$,62813
58,BMW,s,2021,25900$,62813
66,BMW,120,2021,30997$,31700
70,BMW,120,2021,27998$,17370
72,BMW,s,2021,34174$,6731
...,...,...,...,...,...
10511,BMW,i3s,2021,25900$,62813
10539,BMW,i3120,2021,30997$,31700
10553,BMW,i3120,2021,27998$,17370
10562,BMW,i3s,2021,34174$,6731


In [4]:
df['price$'] = df['price$'].apply(lambda x: ('%i$' % x))

In [5]:
df.columns = df.columns.str.capitalize()